<a href="https://colab.research.google.com/github/angelaaaateng/AI_Projects/blob/main/Day5/AI_First_Day_5_Activity_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfplumber
!pip install openai==0.28.1
!pip install tiktoken==0.6.0
!pip install langchain==0.1.20
!pip install chromadb==0.5.0
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB

In [ ]:
import openai
import numpy as np
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding
import faiss
import warnings
import os
warnings.filterwarnings("ignore")
import pdfplumber

In [ ]:
def extract_tables_from_pdf(pdf_path):
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables.extend(page.extract_tables())
    return tables

In [ ]:
tables = extract_tables_from_pdf("/content/Dentist.pdf")
print("Extracted tables :", tables)

Extracted tables : [[['CLINIC NAME', 'CLINIC HOURS', 'CONTACT NO', 'STREET', 'BRGY', 'MUNICIPAL', 'PROVINCE', 'REGION'], ['ALIP DENTAL CLINIC', 'M-S 9:00AM-5:00PM BY APPOINTMENT', '(074) 448-8341, (0917) 627-2610', 'B108 JUNIPER BLDG.,', 'GEN. LUNA RD', 'BAGUIO CITY', 'BENGUET', 'CAR - CORDILLERA ADMINISTRATIVE REGION'], ['SMILE DENTAL CLINIC', 'M-S 9:00AM-5:00PM', '(074) 442-9028', 'UNIT 3-B / 5 & 6 PUSO', 'NSGES BSIAOGNU RIOO,AD', 'BAGUIO CITY', 'BENGUET', 'CAR - CORDILLERA ADMINISTRATIVE REGION'], ['THE DENTAL STUDIO BY: DR. KRISHA SY', 'M-S 9:00AM-6:00PM', '(0917) 506-9031', '2ND FLR. ECCO BLDG.', 'ASSUMPTION ROAD,', 'BAGUIO CITY', 'BENGUET', 'CAR - CORDILLERA ADMINISTRATIVE REGION'], ['GENOVEZA DENTAL CLINIC', 'M-S 9:00AM-6:00PM; SUN (BY APPOINTMENT)', '(074) 423-0947', 'GF 104 BALISONG BLDG', '., ASSUMPTION ROAD.,', 'BAGUIO CITY', 'BENGUET', 'CAR - CORDILLERA ADMINISTRATIVE REGION'], ['DR. ANNA LEAH TORRES-SISON DENTAL CLINIC', 'M-S 10:AM-5:00PM BY APPOINTMENT', '(074) 442-4040, 

In [ ]:
tables

[[['CLINIC NAME',
   'CLINIC HOURS',
   'CONTACT NO',
   'STREET',
   'BRGY',
   'MUNICIPAL',
   'PROVINCE',
   'REGION'],
  ['ALIP DENTAL CLINIC',
   'M-S 9:00AM-5:00PM BY APPOINTMENT',
   '(074) 448-8341, (0917) 627-2610',
   'B108 JUNIPER BLDG.,',
   'GEN. LUNA RD',
   'BAGUIO CITY',
   'BENGUET',
   'CAR - CORDILLERA ADMINISTRATIVE REGION'],
  ['SMILE DENTAL CLINIC',
   'M-S 9:00AM-5:00PM',
   '(074) 442-9028',
   'UNIT 3-B / 5 & 6 PUSO',
   'NSGES BSIAOGNU RIOO,AD',
   'BAGUIO CITY',
   'BENGUET',
   'CAR - CORDILLERA ADMINISTRATIVE REGION'],
  ['THE DENTAL STUDIO BY: DR. KRISHA SY',
   'M-S 9:00AM-6:00PM',
   '(0917) 506-9031',
   '2ND FLR. ECCO BLDG.',
   'ASSUMPTION ROAD,',
   'BAGUIO CITY',
   'BENGUET',
   'CAR - CORDILLERA ADMINISTRATIVE REGION'],
  ['GENOVEZA DENTAL CLINIC',
   'M-S 9:00AM-6:00PM; SUN (BY APPOINTMENT)',
   '(074) 423-0947',
   'GF 104 BALISONG BLDG',
   '., ASSUMPTION ROAD.,',
   'BAGUIO CITY',
   'BENGUET',
   'CAR - CORDILLERA ADMINISTRATIVE REGION'],
  [

In [ ]:
tables[0][0]

['CLINIC NAME',
 'CLINIC HOURS',
 'CONTACT NO',
 'STREET',
 'BRGY',
 'MUNICIPAL',
 'PROVINCE',
 'REGION']

In [ ]:
dataframes = []
dataframed = None
for table in tables:
  dataframed = pd.DataFrame(table[1:], columns=table[0])
  print(dataframed)
  dataframes.append(dataframed)

                                           CLINIC NAME  \
0                                   ALIP DENTAL CLINIC   
1                                  SMILE DENTAL CLINIC   
2                  THE DENTAL STUDIO BY: DR. KRISHA SY   
3                               GENOVEZA DENTAL CLINIC   
4             DR. ANNA LEAH TORRES-SISON DENTAL CLINIC   
..                                                 ...   
161                                  GDH DENTAL CLINIC   
162                                 DENTAL CARE CORNER   
163  DR. FLORENCE A. DE VENECIA DENTAL CLINIC - UCP...   
164                               BUHAIN DENTAL CLINIC   
165                        N. P. RAMOS DENTAL CARE CO.   

                                  CLINIC HOURS  \
0             M-S 9:00AM-5:00PM BY APPOINTMENT   
1                            M-S 9:00AM-5:00PM   
2                            M-S 9:00AM-6:00PM   
3      M-S 9:00AM-6:00PM; SUN (BY APPOINTMENT)   
4              M-S 10:AM-5:00PM BY APPOINTMENT   
.. 

In [ ]:
dataframed.head()

,CLINIC NAME,CLINIC HOURS,CONTACT NO,STREET,BRGY,MUNICIPAL,PROVINCE,REGION
0,ALIP DENTAL CLINIC,M-S 9:00AM-5:00PM BY APPOINTMENT,"(074) 448-8341, (0917) 627-2610","B108 JUNIPER BLDG.,",GEN. LUNA RD,BAGUIO CITY,BENGUET,CAR - CORDILLERA ADMINISTRATIVE REGION
1,SMILE DENTAL CLINIC,M-S 9:00AM-5:00PM,(074) 442-9028,UNIT 3-B / 5 & 6 PUSO,"NSGES BSIAOGNU RIOO,AD",BAGUIO CITY,BENGUET,CAR - CORDILLERA ADMINISTRATIVE REGION
2,THE DENTAL STUDIO BY: DR. KRISHA SY,M-S 9:00AM-6:00PM,(0917) 506-9031,2ND FLR. ECCO BLDG.,"ASSUMPTION ROAD,",BAGUIO CITY,BENGUET,CAR - CORDILLERA ADMINISTRATIVE REGION
3,GENOVEZA DENTAL CLINIC,M-S 9:00AM-6:00PM; SUN (BY APPOINTMENT),(074) 423-0947,GF 104 BALISONG BLDG,"., ASSUMPTION ROAD.,",BAGUIO CITY,BENGUET,CAR - CORDILLERA ADMINISTRATIVE REGION
4,DR. ANNA LEAH TORRES-SISON DENTAL CLINIC,M-S 10:AM-5:00PM BY APPOINTMENT,"(074) 442-4040, (0917) 560-4562","RM. A1, 3/F DOÑA ANIT","A#6 B6L DSEGS.S 2IO,N ROAD",BAGUIO CITY,BENGUET,CAR - CORDILLERA ADMINISTRATIVE REGION


In [ ]:
dataframed['combined'] = dataframed.apply(lambda row : ' '.join(row.values.astype(str)), axis = 1)

In [ ]:
dataframed['combined']

,combined
0,ALIP DENTAL CLINIC M-S 9:00AM-5:00PM BY APPOIN...
1,SMILE DENTAL CLINIC M-S 9:00AM-5:00PM (074) 44...
2,THE DENTAL STUDIO BY: DR. KRISHA SY M-S 9:00AM...
3,GENOVEZA DENTAL CLINIC M-S 9:00AM-6:00PM; SUN ...
4,DR. ANNA LEAH TORRES-SISON DENTAL CLINIC M-S 1...
...,...
161,GDH DENTAL CLINIC M-S 10:00AM-6:00PM BY APPOIN...
162,DENTAL CARE CORNER M-S 10:00AM-7:00PM (02) 928...
163,DR. FLORENCE A. DE VENECIA DENTAL CLINIC - UCP...
164,BUHAIN DENTAL CLINIC M-S 9:00AM-5:00PM STRICTL...


In [ ]:
documents = dataframed['combined'].tolist()
documents

['ALIP DENTAL CLINIC M-S 9:00AM-5:00PM BY APPOINTMENT (074) 448-8341, (0917) 627-2610 B108 JUNIPER BLDG., GEN. LUNA RD BAGUIO CITY BENGUET CAR - CORDILLERA ADMINISTRATIVE REGION',
 'SMILE DENTAL CLINIC M-S 9:00AM-5:00PM (074) 442-9028 UNIT 3-B / 5 & 6 PUSO NSGES BSIAOGNU RIOO,AD BAGUIO CITY BENGUET CAR - CORDILLERA ADMINISTRATIVE REGION',
 'THE DENTAL STUDIO BY: DR. KRISHA SY M-S 9:00AM-6:00PM (0917) 506-9031 2ND FLR. ECCO BLDG. ASSUMPTION ROAD, BAGUIO CITY BENGUET CAR - CORDILLERA ADMINISTRATIVE REGION',
 'GENOVEZA DENTAL CLINIC M-S 9:00AM-6:00PM; SUN (BY APPOINTMENT) (074) 423-0947 GF 104 BALISONG BLDG ., ASSUMPTION ROAD., BAGUIO CITY BENGUET CAR - CORDILLERA ADMINISTRATIVE REGION',
 'DR. ANNA LEAH TORRES-SISON DENTAL CLINIC M-S 10:AM-5:00PM BY APPOINTMENT (074) 442-4040, (0917) 560-4562 RM. A1, 3/F DOÑA ANIT A#6 B6L DSEGS.S 2IO,N ROAD BAGUIO CITY BENGUET CAR - CORDILLERA ADMINISTRATIVE REGION',
 'MOGAN DENTAL CLINIC M-S BY APPOINTMENT (074) 619-8530, (0916) 300-1193 RM 214 NOTRE DAM

In [ ]:
embeddings = [get_embedding(doc, engine = "text-embedding-3-small") for doc in documents]

In [ ]:
embeddings[0][0]

-0.03467756137251854

In [ ]:
embedding_dim = len(embeddings[0])

In [ ]:
embeddings_np = np.array(embeddings).astype('float32')

In [ ]:
embeddings_np

array([[-0.03467756,  0.0138198 ,  0.03503392, ..., -0.01071285,
        -0.00721057,  0.02879774],
       [-0.01134395,  0.02493467,  0.01550707, ...,  0.01262152,
        -0.00552054,  0.02197202],
       [-0.01422601,  0.0102858 ,  0.02787932, ..., -0.00178684,
        -0.01633357,  0.02019361],
       ...,
       [-0.04251321,  0.01499734,  0.04649741, ...,  0.02988149,
         0.01435217,  0.01558591],
       [-0.03173824,  0.03425085,  0.05999409, ...,  0.01895478,
        -0.02167678,  0.00347688],
       [ 0.00186082,  0.04756372,  0.07765405, ...,  0.00428903,
         0.03367775,  0.01469131]], dtype=float32)

In [ ]:
index = faiss.IndexFlatL2(embedding_dim)

In [ ]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x79423be6b8d0> >

In [ ]:
index.add(embeddings_np)

In [ ]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x79423be6b8d0> >

In [ ]:
user_message = "Where is the Alip Dental Clinic?"

In [ ]:
query_embedding = get_embedding(user_message, engine='text-embedding-3-small')
query_embedding_np = np.array([query_embedding]).astype('float32')

In [ ]:
_, indices = index.search(query_embedding_np, 1)

In [ ]:
indices

array([[0]])

In [ ]:
retrieved_docs = [documents[i] for i in indices[0]]

In [ ]:
retrieved_docs

['ALIP DENTAL CLINIC M-S 9:00AM-5:00PM BY APPOINTMENT (074) 448-8341, (0917) 627-2610 B108 JUNIPER BLDG., GEN. LUNA RD BAGUIO CITY BENGUET CAR - CORDILLERA ADMINISTRATIVE REGION']

In [ ]:
context = ' '.join(retrieved_docs)

In [ ]:
structured_prompt = f"Context:\n{context}\n\nQuery:\n{user_message}\n\nResponse:"

In [ ]:
print(structured_prompt)

Context:
ALIP DENTAL CLINIC M-S 9:00AM-5:00PM BY APPOINTMENT (074) 448-8341, (0917) 627-2610 B108 JUNIPER BLDG., GEN. LUNA RD BAGUIO CITY BENGUET CAR - CORDILLERA ADMINISTRATIVE REGION

Query:
Where is the Alip Dental Clinic?

Response:


In [ ]:
struct = [{"role": "system", "content": "You are a helpful Assistant"}]

In [ ]:
chat =  openai.ChatCompletion.create(model = "gpt-4o-mini", messages = struct + [{"role": "user", "content" : structured_prompt}], temperature=0.5, max_tokens=1500, top_p=1, frequency_penalty=0, presence_penalty=0)
struct.append({"role": "user", "content": user_message})
response = chat.choices[0].message.content
struct.append({"role": "assistant", "content": response})

In [ ]:
print(response)

The Alip Dental Clinic is located at B108 Juniper Building, Gen. Luna Road, Baguio City, Benguet, in the Cordillera Administrative Region of the Philippines.


In [ ]:
user_message = "What clinics are in makati?"
query_embedding = get_embedding(user_message, engine='text-embedding-3-small')
query_embedding_np = np.array([query_embedding]).astype('float32')
_, indices = index.search(query_embedding_np, 1)
retrieved_docs = [documents[i] for i in indices[0]]
context = ' '.join(retrieved_docs)
structured_prompt = f"Context:\n{context}\n\nQuery:\n{user_message}\n\nResponse:"

In [ ]:
chat =  openai.ChatCompletion.create(model = "gpt-4o-mini",
                                     messages = struct + [{"role": "user", "content" : structured_prompt}],
                                     temperature=0.5, max_tokens=1500,
                                     top_p=1, frequency_penalty=0, presence_penalty=0)
struct.append({"role": "user", "content": user_message})
response = chat.choices[0].message.content
struct.append({"role": "assistant", "content": response})
print(response)

Here are some dental clinics located in Makati:

1. **Makati Dental Clinic**
   - Services: General dentistry, orthodontics, cosmetic dentistry
   - Location: Various branches in Makati

2. **The Dental Room**
   - Address: 2/F, The Residences at Greenbelt, 106 A. Arnaiz Ave, Makati
   - Services: Comprehensive dental services, cosmetic dentistry

3. **Makati Medical Center Dental Clinic**
   - Address: 2 Amorsolo St, Legaspi Village, Makati
   - Services: Dental surgery, orthodontics, pediatric dentistry

4. **Dental Health and Wellness Center**
   - Address: 2/F, Makati Square, Chino Roces Ave, Makati
   - Services: General dentistry, preventive care

5. **Smile Avenue Dental Clinic**
   - Address: 2/F, SM City Makati, Makati
   - Services: Family dentistry, cosmetic procedures

6. **Clinica Dental de Manila - Makati Branch**
   - Address: 2/F, The Fort Strip, Bonifacio Global City, Makati
   - Services: General and cosmetic dentistry

Make sure to call ahead to confirm services and 